In [7]:
# https://kanoki.org/2019/02/26/compare-two-excel-files-for-difference-using-python/

# Stock trade verification code

import numpy as np
import pandas as pd

In [8]:
oms_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/20230530_oms.xls", )
trader_df = pd.read_excel("Z:/02.펀드/019. 일간매매내역/5월30일 거래.xlsx", )

In [9]:
# https://sparkbyexamples.com/pandas/pandas-select-columns-by-name-or-index/
# 필요한 열 선택



In [10]:
# Short sell, Buy cover, Buy, Sell 매매구분을 위한 열 병합 및 값 변환
oms_df1 = oms_df[["펀드", "종목명", "매매유형", "매매구분", "체결수량", "체결단가", "체결금액"]]

oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)

oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])

oms_df1 = oms_df1.drop(oms_df1.columns[[2]], axis=1)

# oms_df1 = oms_df1.replace(1, np.nan)
oms_df1 = oms_df1.dropna()

col_to_move = oms_df1.pop('매매구분')

oms_df1.insert(2, '매매구분', col_to_move)

C:\Users\jtlee\AppData\Local\Temp\ipykernel_17404\1968330389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분"]].agg('_'.join, axis=1)
C:\Users\jtlee\AppData\Local\Temp\ipykernel_17404\1968330389.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])


#둘 다 키값을 종목명(Key1) and 체결수량(Key2)으로 잡고 정렬한 다음 데이터프레임의 밸류들이 일치하는지 확인

In [11]:
oms_df1

,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,공모주1호,네패스,Buy cover,200,20652,4130350
1,공모주1호,대주전자재료,Buy cover,300,90905,27271600
2,공모주1호,KODEX코스닥150선물인버스,Buy,25000,4098,102450000
3,공모주1호,쏘카,Sell,1290,16156,20841530
4,공모주1호,제이오,Buy,1000,31055,31055000
5,공모주1호,기가비스,Buy,300,71096,21328700
6,코스닥벤처,POSCO홀딩스,Buy cover,200,367188,73437500
7,코스닥벤처,NAVER,Short sell,450,202000,90900000
8,코스닥벤처,KH바텍,Buy cover,2000,21163,42326100
9,코스닥벤처,대주전자재료,Buy cover,230,91379,21017100


In [12]:
oms_df2 = oms_df1.set_index(['펀드', '종목명', '매매구분'])

oms_df3 = oms_df2.sort_index(axis=0)

oms_df3 = oms_df3[(oms_df3 != 0).all(axis=1)]

oms_df4 = oms_df3.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})

In [13]:
trader_df1 = trader_df[["펀드", "종목명", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [14]:
trader_df1 = trader_df1.dropna()

trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')

trader_df3 = trader_df2.groupby(['펀드', '종목명', '매매구분']).agg({'체결수량': 'sum', '체결단가': 'mean', '체결금액': 'sum'})/

C:\Users\jtlee\AppData\Local\Temp\ipykernel_17404\2518177081.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  trader_df2 = trader_df1.set_index('펀드', '종목명', '매매구분')


In [15]:
trader_df3 = trader_df3.sort_index(axis=0)

In [16]:
trader_df3 = trader_df3.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})
oms_df4 = oms_df4.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})

In [17]:
trader_df3['체결금액'] = trader_df3['체결금액'].abs()

trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSPI200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != 'KOSDAQ150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피200F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥150F']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스피F200']
except:
    pass
try:
    trader_df3 = trader_df3[trader_df3['종목명'] != '코스닥F150']
except:
    pass

In [18]:
# trader_df3.compare(oms_df3)

In [19]:
trader_df3 = trader_df3.reset_index()
oms_df4 = oms_df4.reset_index()

In [20]:
trader_df3

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,25000,4098,102450000
1,1,공모주1호,기가비스,Buy,300,71096,21328800
2,2,공모주1호,네패스,Buy cover,200,20652,4130400
3,3,공모주1호,대주전자재료,Buy cover,300,90905,27271500
4,4,공모주1호,쏘카,Sell,1290,16156,20841240
5,5,공모주1호,제이오,Buy,1000,31055,31055000
6,6,멀티전략,ISC,Buy cover,1000,52058,52058000
7,7,멀티전략,KH바텍,Buy cover,2000,21105,42210000
8,8,멀티전략,KODEX200선물인버스2x,Buy,50000,2515,125750000
9,9,멀티전략,KODEX코스닥150선물인버스,Buy,25000,4098,102450000


In [21]:
oms_df4

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,25000,4098,102450000
1,1,공모주1호,기가비스,Buy,300,71096,21328700
2,2,공모주1호,네패스,Buy cover,200,20652,4130350
3,3,공모주1호,대주전자재료,Buy cover,300,90905,27271600
4,4,공모주1호,쏘카,Sell,1290,16156,20841530
5,5,공모주1호,제이오,Buy,1000,31055,31055000
6,6,멀티전략,ISC,Buy cover,1000,52058,52058100
7,7,멀티전략,KH바텍,Buy cover,2000,21105,42209800
8,8,멀티전략,KODEX200선물인버스2X,Buy,50000,2515,125750000
9,9,멀티전략,KODEX코스닥150선물인버스,Buy,25000,4098,102450000


In [22]:
result_df = oms_df4

In [23]:
result_df[['종목명']] = oms_df4[['종목명']]
result_df[['매매구분']] = oms_df4[['매매구분']]
result_df[['체결수량']] = oms_df4[['체결수량']] - trader_df3[['체결수량']]
result_df[['체결단가']] = oms_df4[['체결단가']] - trader_df3[['체결단가']]
result_df[['체결금액']] = oms_df4[['체결금액']] - trader_df3[['체결금액']]

In [24]:
result_df

,index,펀드,종목명,매매구분,체결수량,체결단가,체결금액
0,0,공모주1호,KODEX코스닥150선물인버스,Buy,0,0,0
1,1,공모주1호,기가비스,Buy,0,0,-100
2,2,공모주1호,네패스,Buy cover,0,0,-50
3,3,공모주1호,대주전자재료,Buy cover,0,0,100
4,4,공모주1호,쏘카,Sell,0,0,290
5,5,공모주1호,제이오,Buy,0,0,0
6,6,멀티전략,ISC,Buy cover,0,0,100
7,7,멀티전략,KH바텍,Buy cover,0,0,-200
8,8,멀티전략,KODEX200선물인버스2X,Buy,0,0,0
9,9,멀티전략,KODEX코스닥150선물인버스,Buy,0,0,0
